# Prompting

In [3]:
import openai
import os

api_key = os.getenv("OPENAI_API_KEY")
organization = os.getenv("OPEN_AI_ORG")
# elevenlabs_key = os.getenv("ELEVENLABS_KEY")

## Chain-of-Thought (CoT)
<img src="../figures/cot-prompting.png" >

In [15]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

examples = [
    {"input": "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?", 
     "output": "Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11."},
]

# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?
AI: Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11.


In [16]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [17]:
from langchain.chat_models import ChatOpenAI

chain = final_prompt | ChatOpenAI(
    temperature=0,
    openai_api_key=api_key,
    openai_organization=organization,
    )

chain.invoke({"input": "The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"})

AIMessage(content='The cafeteria started with 23 apples. They used 20 for lunch, so they have 23 - 20 = 3 apples left. Then they bought 6 more, so they have 3 + 6 = 9 apples now. The answer is 9.')

## Zero-shot CoT

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

chat_template = ChatPromptTemplate.from_messages(
    [
        ("human", "{input} Let's think step by step."),

    ]
)

chain = chat_template | ChatOpenAI(
    temperature=0,
    openai_api_key=api_key,
    openai_organization=organization,
    )

chain.invoke({"input" : "The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"})

AIMessage(content='Step 1: Start with the initial number of apples in the cafeteria, which is 23.\n\nStep 2: Subtract the number of apples used to make lunch, which is 20. \n\n23 - 20 = 3\n\nStep 3: Add the number of apples bought, which is 6.\n\n3 + 6 = 9\n\nTherefore, the cafeteria now has 9 apples.')

## Tree-of-Thought (ToT)

The Tree of Thought (ToT) is a chain that allows you to query a Large Language Model (LLM) using the Tree of Thought technique. 
This is based on the paper [\"Large Language Model Guided Tree-of-Thought\"](https://arxiv.org/pdf/2305.08291.pdf)"

### Using Sequential Chain

In [7]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI


template ="""
Step1 :
 
I have a problem related to {input}. Could you brainstorm three distinct solutions? Please consider a variety of factors such as {perfect_factors}
A:
"""

prompt1 = PromptTemplate(
    input_variables=["input","perfect_factors"],
    template = template                      
)

chain1 = LLMChain(
    llm=ChatOpenAI(
        temperature=0,
        openai_api_key=api_key,
        openai_organization=organization,
        ),
    prompt=prompt1,
    output_key="solutions",
    verbose=True
)

In [8]:
template ="""
Step 2:

For each of the three proposed solutions, evaluate their potential. Consider their pros and cons, initial effort needed, implementation difficulty, potential challenges, and the expected outcomes. Assign a probability of success and a confidence level to each option based on these factors

{solutions}

A:"""

prompt2 = PromptTemplate(
    input_variables=["solutions"],
    template = template                      
)

chain2 = LLMChain(
    llm=ChatOpenAI(
        temperature=0,
        openai_api_key=api_key,
        openai_organization=organization,
        ),
    prompt=prompt2,
    output_key="review",
    verbose=True
)

In [9]:
template ="""
Step 3:

For each solution, deepen the thought process. Generate potential scenarios, strategies for implementation, any necessary partnerships or resources, and how potential obstacles might be overcome. Also, consider any potential unexpected outcomes and how they might be handled.

{review}

A:"""

prompt3 = PromptTemplate(
    input_variables=["review"],
    template = template                      
)

chain3 = LLMChain(
    llm=ChatOpenAI(
        temperature=0,
        openai_api_key=api_key,
        openai_organization=organization,
        ),
    prompt=prompt3,
    output_key="deepen_thought_process",
    verbose=True
)


In [10]:
template ="""
Step 4:

Based on the evaluations and scenarios, rank the solutions in order of promise. Provide a justification for each ranking and offer any final thoughts or considerations for each solution
{deepen_thought_process}

A:"""

prompt4 = PromptTemplate(
    input_variables=["deepen_thought_process"],
    template = template                      
)

chain4 = LLMChain(
    llm=ChatOpenAI(
        temperature=0,
        openai_api_key=api_key,
        openai_organization=organization,
        ),
    prompt=prompt4,
    output_key="ranked_solutions",
    verbose=True
)

In [11]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["input", "perfect_factors"],
    output_variables=["ranked_solutions"],
    verbose=True
)


reuslt = overall_chain({
        "input": "human colonization of Mars", 
        "perfect_factors": "The distance between Earth and Mars is very large, making regular resupply difficult"
    })




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Step1 :
 
I have a problem related to human colonization of Mars. Could you brainstorm three distinct solutions? Please consider a variety of factors such as The distance between Earth and Mars is very large, making regular resupply difficult
A:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Step 2:

For each of the three proposed solutions, evaluate their potential. Consider their pros and cons, initial effort needed, implementation difficulty, potential challenges, and the expected outcomes. Assign a probability of success and a confidence level to each option based on these factors

1. Sustainable Resource Management: One solution could be to focus on sustainable resource management on Mars. This would involve developing technologies and systems that allow for efficient use of available resources, such as water, energy, and food. By maximizing reso

In [12]:
reuslt.keys()

dict_keys(['input', 'perfect_factors', 'ranked_solutions'])

In [13]:
print(reuslt['ranked_solutions'])

Rank: 1

Justification: Vertical farming has the highest promise due to its potential to reduce dependence on resupply from Earth, maximize food production in a limited space, and provide fresh and nutritious food for the colony. While there are initial investment and implementation challenges, the probability of success is high, and the confidence level is also high. The potential scenarios, strategies for implementation, partnerships and resources, and overcoming potential obstacles all support the feasibility and potential success of vertical farming.

B: Rank: 2

Justification: The second-highest promise solution is the development of advanced water recycling systems. While it is crucial for sustaining life on Mars, it is not as promising as vertical farming in terms of reducing dependence on Earth for food. However, it is still a critical solution for ensuring a sustainable water supply for the colony. The potential scenarios, strategies for implementation, partnerships and resour

### Using ToTChain

In [4]:
sudoku_puzzle =   "3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1"
sudoku_solution = "3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1"
problem_description = f"""
{sudoku_puzzle}

- This is a 4x4 Sudoku puzzle.
- The * represents a cell to be filled.
- The | character separates rows.
- At each step, replace one or more * with digits 1-4.
- There must be no duplicate digits in any row, column or 2x2 subgrid.
- Keep the known digits from previous valid thoughts in place.
- Each thought can be a partial or the final solution.
""".strip()

print(problem_description)

3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1

- This is a 4x4 Sudoku puzzle.
- The * represents a cell to be filled.
- The | character separates rows.
- At each step, replace one or more * with digits 1-4.
- There must be no duplicate digits in any row, column or 2x2 subgrid.
- Keep the known digits from previous valid thoughts in place.
- Each thought can be a partial or the final solution.


In [5]:
from langchain_experimental.tot.base import ToTChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_experimental.tot.checker import ToTChecker
from langchain_experimental.tot.thought import ThoughtValidity
import re
from typing import Tuple

class MyChecker(ToTChecker):
    def evaluate(self, problem_description: str, thoughts: Tuple[str, ...] = ()) -> ThoughtValidity:
        print(thoughts)
        last_thought = thoughts[-1]
        clean_solution = last_thought.replace(" ", "").replace('"', "")
        regex_solution = clean_solution.replace("*", ".").replace("|", "\\|")
        if sudoku_solution in clean_solution:
            return ThoughtValidity.VALID_FINAL
        elif re.search(regex_solution, sudoku_solution):
            return ThoughtValidity.VALID_INTERMEDIATE
        else:
            return ThoughtValidity.INVALID

#######
# Testing the MyChecker class above:
#######

checker = MyChecker()
assert checker.evaluate("", ("3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1",)) == ThoughtValidity.VALID_INTERMEDIATE
assert checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1",)) == ThoughtValidity.VALID_FINAL
assert checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,3,*,1",)) == ThoughtValidity.VALID_INTERMEDIATE
assert checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,*,3,1",)) == ThoughtValidity.INVALID

('3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1',)
('3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1',)
('3,4,1,2|1,2,3,4|2,1,4,3|4,3,*,1',)
('3,4,1,2|1,2,3,4|2,1,4,3|4,*,3,1',)


In [6]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0,
    openai_api_key=api_key,
    openai_organization=organization,
)

tot_chain = ToTChain(
    llm = llm,
    memory = ConversationBufferWindowMemory(),
    checker = checker,
    verbose = True,
    # verbose_llm = True
)

tot_chain.run({"problem_description": problem_description})



> Entering new ToTChain chain...
Starting the ToT solve procedure.


/opt/homebrew/lib/python3.11/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


('3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1',)
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1


/opt/homebrew/lib/python3.11/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


('3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1', '3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1')
    Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1


...

/opt/homebrew/lib/python3.11/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


('3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1', '3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1', '3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1', '3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1', '3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1')
                Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1


/opt/homebrew/lib/python3.11/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


('3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1', '3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1', '3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1', '3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1', '3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1', '3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1')
                    Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1


/opt/homebrew/lib/python3.11/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


# Reference 
https://medium.com/@astropomeai/implementing-the-tree-of-thoughts-in-langchains-chain-f2ebc5864fac
https://github.com/mrspiggot/forestOfThoughts/blob/master/langchain_monoculture.py
https://cobusgreyling.medium.com/langchain-langsmith-llm-guided-tree-of-thought-47a2cd5bcfca